In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
count=20
data=pd.read_csv(f'cluster/hourly_{count}.csv')

data=data.drop(columns=['station_id','1小时雨量','6小时雨量','date_y','is_holiday'])


data = data.dropna()  
X = data.drop(columns=['date', 'PM25_Concentration'])  # 事件变量
y = data['PM25_Concentration']  

split_idx = int(0.8 * len(data))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

param_grid = {
    'max_depth': [10],
    'learning_rate': [0.05],
    'n_estimators': [100],
    'reg_alpha': [0.1],  
    'reg_lambda': [0.9]   
}

tscv = TimeSeriesSplit(n_splits=5)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search = GridSearchCV(xgb_model, param_grid, cv=tscv, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.2f} MSE: {mse:.2f} MAE: {mae:.2f}, R²: {r2:.2f}")

explainer = shap.TreeExplainer(best_model)
shap_values = explainer(X_test)

shap_df = pd.DataFrame({
    'event': X.columns,
    'shap_impact': np.abs(shap_values.values).mean(axis=0),
    'direction': np.where(shap_values.values.mean(axis=0) > 0, '+', '-')
}).sort_values('shap_impact', ascending=False)
shap_values_full = explainer(data.drop(columns=['date', 'PM25_Concentration']))
print("Top 30 影响事件:")
print(shap_df.head(31))


In [ ]:
event_columns = [
    "Traffic Accidents", "Transportation Construction and Development",
    "Transportation Rehabilitation and Adjustment", "Traffic Control",
    "Humanities Activities", "Urban Management", "Large Social Events",
    "Large Examinations", "Accidents", "Extreme Weather", "Temperature Changes",
    "Meteorological Transformations", "Meteorological Publicity",
    "Epidemic Prevention and Control", "Adjustment of urban operations",
    "Disease Outbreaks", "Social Security Incidents", "Air Pollution",
    "Aviation Safety Incidents", "Illegal Incidents"
]

event_df = shap_df[shap_df['event'].isin(event_columns)]

event_df['impact_ratio'] = event_df['shap_impact'] / event_df['shap_impact'].sum()

filtered_shap_df = event_df.sort_values(by='impact_ratio', ascending=False)

print("\n每个指定事件在所有事件中影响占比：")
for idx, row in filtered_shap_df.iterrows():
    print(f"{row['event']}: {row['impact_ratio']:.4f}")

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
count=100
data=pd.read_csv(f'cluster/hourly_{count}.csv')
data = data[data['station_id'] == '1001A']

data=data.drop(columns=['station_id','1小时雨量','6小时雨量'])

data = data.dropna()  
X = data.drop(columns=['date', 'PM25_Concentration'])  # 事件变量
y = data['PM25_Concentration']  

split_idx = int(0.8 * len(data))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

param_grid = {
    'max_depth': [10],
    'learning_rate': [0.05],
    'n_estimators': [100],
    'reg_alpha': [0.1], 
    'reg_lambda': [0.9]  
}

tscv = TimeSeriesSplit(n_splits=5)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search = GridSearchCV(xgb_model, param_grid, cv=tscv, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.2f} MSE: {mse:.2f} MAE: {mae:.2f}, R²: {r2:.2f}")

explainer = shap.TreeExplainer(best_model)
shap_values = explainer(X_test)
shap_df = pd.DataFrame({
    'event': X.columns,
    'shap_impact': np.abs(shap_values.values).mean(axis=0),
    'direction': np.where(shap_values.values.mean(axis=0) > 0, '+', '-')
}).sort_values('shap_impact', ascending=False)
shap_values_full = explainer(data.drop(columns=['date', 'PM25_Concentration']))
print("Top 30 影响事件:")
print(shap_df.head(31))

In [ ]:
event_columns = [
    "Haze Weather",
    "Thunderstorm Weather",
    "Epidemic Prevention",
    "COVID-19 Epidemic Prevention",
    "High Temperature Weather",
    "Work Resumption and Production Recovery",
    "Cold Wave Weather",
    "Heating Supply Activities",
    "COVID-19 New Cases",
    "Public Order Issues",
    "Heavy Rain Weather",
    "Campus Incidents",
    "Frequent Respiratory Infections",
    "Cultural Activities",
    "Criminal Cases",
    "Pandemic Intensification",
    "Traffic Accidents",
    "Traffic Adjustment and Optimization",
    "College Entrance Exam",
    "Performances"
]

# 筛选出指定事件列
event_df = shap_df[shap_df['event'].isin(event_columns)]

# 计算每个事件的影响占比
event_df['impact_ratio'] = event_df['shap_impact'] / event_df['shap_impact'].sum()

# 按照影响占比降序排序
filtered_shap_df = event_df.sort_values(by='impact_ratio', ascending=False)

# 输出结果
print("\n每个指定事件在所有事件中影响占比：")
for idx, row in filtered_shap_df.iterrows():
    print(f"{row['event']}: {row['impact_ratio']:.4f}")


In [ ]:
selected_features=event_columns
feature_indices = [X.columns.get_loc(col) for col in selected_features]
shap_values_selected = shap_values[:, feature_indices]
X_test_selected = X_test.iloc[:, feature_indices]
shap.summary_plot(shap_values_selected, 
                 X_test_selected, 
                 plot_type="bar", 
                 feature_names=selected_features,
                 show=False)  
plt.xlabel('mean|SHAP|')

plt.show()

shap.summary_plot(shap_values_selected, 
                 X_test_selected, 
                 feature_names=selected_features,
                 show=False)

plt.show()

In [ ]:
print(X_test.columns)
for feature in event_columns:
    if feature in X_test.columns:  # 确保特征存在
        shap.dependence_plot(feature, shap_values.values, X_test, interaction_index=None)
    else:
        print(f"警告: 特征 {feature} 不在 X_test 中，跳过绘制")